# YOUR PROJECT TITLE

Vi skal: 

1. ændre percentiler til niveauer, og lav standardisering

2. forklar BNP og ulighed på baggrund af governance. 
    - lag governance evt. 10 år.

3. Find varians i BNP og forklar på baggrund af (lagged værdi af) governance

4. Figurer... evt: 
    - Scatterplot over vækst ift. ulighed. 
    - andet?

Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


# Read and clean data

In [2]:
wb_gini = wb.download(country='all', indicator='SI.POV.GINI', start=1990, end=2017)
wb_gini = wb_gini.rename(columns = {'SI.POV.GINI':'GINI'})
wb_gini = wb_gini.reset_index()
wb_gini.year = wb_gini.year.astype(int) # convert year
wb_gini.country = wb_gini.country.astype('string') # convert country to the special pandas string type
wb_gini.info()

wb_gdp = wb.download(country='all', indicator='NY.GDP.PCAP.KD', start=1990, end=2017)
wb_gdp = wb_gdp.rename(columns = {'NY.GDP.PCAP.KD':'GDP'})
wb_gdp.reset_index(inplace = True)
wb_gdp.year = wb_gdp.year.astype(int)
wb_gdp.country = wb_gdp.country.astype('string') # convert country to the special pandas string type
wb_gdp.info()
wb_gdp.head()

wb = pd.merge(wb_gini, wb_gdp, how = 'outer', on = ['country','year']);
wb.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GINI     1582 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GDP      6772 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB


,country,year,GINI,GDP
0,Africa Eastern and Southern,2017,NaN,1534.683482
1,Africa Eastern and Southern,2016,NaN,1534.924746
2,Africa Eastern and Southern,2015,NaN,1538.552268
3,Africa Eastern and Southern,2014,NaN,1537.159571
4,Africa Eastern and Southern,2013,NaN,1518.482850


In [3]:
wb_wgi=pd.read_csv("WB WGI Levels.csv")

# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3'
drop_these = ['Country Code','Series Code'] # use list comprehension to create list of columns
wb_wgi.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
wb_wgi.head(5)

col_dict = {}
for i in range(1996, 2022): # range goes from 1996 to 2021
    col_dict[str(i)+" [YR"+str(i)+"]"] = f'wgi{i}' 
col_dict

wb_wgi.rename(columns = col_dict, inplace=True)
wb_wgi.rename (columns ={'Country Name': 'country'}, inplace=True)
wb_wgi.sample(10)

,country,Series Name,wgi1996,wgi1998,wgi2000,wgi2002,wgi2003,wgi2004,wgi2005,wgi2006,...,wgi2012,wgi2013,wgi2014,wgi2015,wgi2016,wgi2017,wgi2018,wgi2019,wgi2020,wgi2021
100,Bangladesh,Rule of Law: Estimate,-0.93402373790741,-0.935864508152008,-0.991129994392395,-0.991601228713989,-1.15194392204285,-1.12248206138611,-0.978551864624023,-0.907564043998718,...,-0.936167359352112,-0.87175977230072,-0.782509088516235,-0.759843051433563,-0.65733277797699,-0.665030121803284,-0.633198857307434,-0.631936311721802,-0.560531854629517,-0.611260771751404
159,Botswana,Regulatory Quality: Estimate,0.650729417800903,0.832761347293854,0.796799182891846,0.900155961513519,0.844509065151215,0.763184726238251,0.720574915409088,0.610028147697449,...,0.692556440830231,0.635518550872803,0.764807999134064,0.56475830078125,0.736300766468048,0.60569304227829,0.632988631725311,0.509274005889893,0.53995269536972,0.614231944084167
1057,South Sudan,Government Effectiveness: Estimate,..,..,..,..,..,..,..,..,...,-1.62352359294891,-1.68720555305481,-2.10063147544861,-2.1370005607605,-2.33561444282532,-2.45031404495239,-2.32946562767029,-2.37982869148254,-2.31318593025208,-2.38260555267334
955,Romania,Government Effectiveness: Estimate,-0.286933869123459,-0.52674263715744,-0.262158989906311,-0.196587815880775,-0.289749026298523,-0.180373430252075,-0.295782208442688,-0.194396048784256,...,-0.171900972723961,0.136252075433731,0.226028010249138,0.128101199865341,-0.0402308702468872,-0.0583919622004032,-0.147446274757385,-0.186415791511536,-0.257351338863373,-0.129637852311134
134,Bermuda,Political Stability and Absence of Violence/Te...,0.717076897621155,0.728511989116669,0.713235020637512,0.792559921741486,0.851720035076141,0.893765926361084,0.819073557853699,0.77465146780014,...,0.98025369644165,0.970303058624268,0.913897812366486,0.983049035072327,0.999670088291168,1.00191104412079,1.15247452259064,1.06507384777069,1.00745189189911,1.0061514377594
732,Martinique,Control of Corruption: Estimate,0.869897186756134,0.883366405963898,0.861537575721741,0.85021036863327,0.854120075702667,0.885075032711029,0.789309144020081,0.775587141513824,...,0.77997100353241,0.77830171585083,1.22070860862732,1.21544027328491,1.22331142425537,1.23786544799805,1.23453342914581,1.23104310035706,0.675268352031708,0.68094277381897
454,Grenada,Rule of Law: Estimate,0.425557196140289,0.421897411346436,0.341131985187531,0.359068334102631,0.337395370006561,0.280416071414948,0.378238171339035,0.259322911500931,...,0.186642453074455,0.199680164456367,-0.137727960944176,0.209968268871307,0.653643786907196,0.491692453622818,0.280941128730774,0.208473965525627,0.35385924577713,0.511254489421844
1056,South Sudan,Control of Corruption: Estimate,..,..,..,..,..,..,..,..,...,-1.29581928253174,-1.400599360466,-1.6076123714447,-1.70206952095032,-1.73977863788605,-1.73264741897583,-1.72504222393036,-1.78442513942719,-1.91645681858063,-1.81744623184204
600,"Korea, Dem. People's Rep.",Control of Corruption: Estimate,-1.28434658050537,-1.24869453907013,-1.30290651321411,-1.02666258811951,-1.43155789375305,-1.38584494590759,-1.28446447849274,-1.4435168504715,...,-1.28702855110168,-1.29103326797485,-1.30408978462219,-1.25436878204346,-1.44655621051788,-1.48626208305359,-1.57093775272369,-1.5939404964447,-1.6856027841568,-1.6050671339035
982,San Marino,Rule of Law: Estimate,0.809140563011169,0.876508176326752,0.839712262153625,0.83394581079483,0.833284139633179,0.826330423355103,0.776517033576965,0.984307050704956,...,0.926880657672882,0.92919784784317,..,..,..,..,..,..,..,..


In [4]:
wb_wgi = pd.wide_to_long(wb_wgi, stubnames='wgi', i=['country','Series Name'], j='year')
wb_wgi = wb_wgi.reset_index()
# Rename colum with series names to "ser"
wb_wgi.rename(columns = {'Series Name':'ser'}, inplace=True)

#Get a list of all the series in the data
namelist = wb_wgi.ser.unique()

newnames = {'Control of Corruption: Estimate':'COC',
            'Government Effectiveness: Estimate':'GOV', 
            'Political Stability and Absence of Violence/Terrorism: Estimate':'RSA', 
            'Regulatory Quality: Estimate':'REQ', 
            'Rule of Law: Estimate':'ROL', 
            'Voice and Accountability: Estimate':'VOA'}

# Renames series
for index, name in enumerate(namelist):
    print(index, "Now rename Series", name, "=", newnames[name])
    
    wb_wgi.loc[wb_wgi.ser == name, 'ser'] = newnames[name]

newnamelist = wb_wgi.ser.unique()

wb_wgi.sample(5)

0 Now rename Series Control of Corruption: Estimate = COC
1 Now rename Series Government Effectiveness: Estimate = GOV
2 Now rename Series Political Stability and Absence of Violence/Terrorism: Estimate = RSA
3 Now rename Series Regulatory Quality: Estimate = REQ
4 Now rename Series Rule of Law: Estimate = ROL
5 Now rename Series Voice and Accountability: Estimate = VOA


,country,ser,year,wgi
3511,Bosnia and Herzegovina,RSA,2014,-0.0191790424287319
358,Algeria,REQ,2012,-1.32038903236389
22619,San Marino,VOA,2009,1.1825864315033
18117,Morocco,GOV,2015,-0.111680008471012
23565,Singapore,ROL,2012,1.72407805919647


In [5]:
wb_new=pd.pivot(wb_wgi, index=['country','year'], columns = 'ser', values= 'wgi')
wb_new=wb_new.reset_index()
final = pd.merge(wb, wb_new, on=['year', 'country'], how = 'outer')

In [7]:
col_list = ['COC', 'GOV', 'REQ', 'ROL', 'RSA', 'VOA']
for i in col_list:
    final.loc[final[i]==".."] = np.nan
    final[i]=final[i].astype(float)
final = final.dropna(subset=['country','year'], how='any')
final.year = final.year.astype(int)
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8191 entries, 0 to 8493
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  8191 non-null   object 
 1   year     8191 non-null   int32  
 2   GINI     1577 non-null   float64
 3   GDP      6624 non-null   float64
 4   COC      4619 non-null   float64
 5   GOV      4619 non-null   float64
 6   REQ      4619 non-null   float64
 7   ROL      4619 non-null   float64
 8   RSA      4619 non-null   float64
 9   VOA      4619 non-null   float64
dtypes: float64(8), int32(1), object(1)
memory usage: 671.9+ KB


C:\Users\krist\AppData\Local\Temp\ipykernel_18696\359575070.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.year = final.year.astype(int)


#given variabel: træk gennemsnittet fra og divider med standardafvigelsen

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.